<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/movie_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
・GPUを使用するのでランタイムのタイプの変更からGPUに設定する必要がある
"""

'\n・GPUを使用するのでランタイムのタイプの変更からGPUに設定する必要がある\n'

In [ ]:
!pip install "faster-whisper @ https://github.com/nyanta012/faster-whisper/archive/refs/heads/master.tar.gz"
!pip install transformers==4.27.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 13.5 kB 48.7 kB/s 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.7/31.7 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
  Created wheel for faster-whisper: filename=faster_whisper-0.1.0-py3-none-any.whl size=13988 sha256=5020b5b46b683bbac519fefb8741532ac5b54abc9cc4897f4bd795cb70bf1884
  Stored in directory: /tmp/pip-ephem-wheel-cache-ws4bc_4b/wheels/e7/6c/49/8cf262ab990e0f9835ae16806f9138d17376028f08132ec7b9
Successfully built faster-whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.9 MB/s eta 0:00:00


In [ ]:
# whisper large-v2をhugging faceから取得して、ctranslate2でC++に変換&パラメータなどの量子化する
from faster_whisper import WhisperModel

!ct2-transformers-converter --model openai/whisper-large-v2 --output_dir whisper-large-v2-ct2 \
    --copy_files tokenizer.json --quantization float16

model_path = "whisper-large-v2-ct2/"

# Run on GPU with FP16
model = WhisperModel(model_path, device="cuda", compute_type="float16")

In [ ]:
#text file を作成するための関数
def create_txt_file(file_name="transcribe", results=None, fast_whisper=True):
    with open(f"{file_name}.txt", mode="w") as f:
        for index, _dict in enumerate(results):
            if fast_whisper:
              start_time = _dict[0] # start
              end_time = _dict[1] # end
              text = _dict[2] # text
            else:
              start_time = _dict["start"]
              end_time = _dict["end"]
              text = _dict["text"]
            s_h, e_h = int(start_time//(60 * 60)), int(end_time//(60 * 60))
            s_m, e_m = int((start_time % (60 * 60))//60), int((end_time % (60 * 60))//60)
            s_s, e_s = int(start_time % 60), int(end_time % 60)
            f.write(f'{text}\n')

In [ ]:
# google driveをマウントする
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir("/content/drive/MyDrive/")

['Colab Notebooks', 'youtube', 'ReazonSpeech.mp4', 'old']

In [ ]:
# 文字起こしを行い、textファイルを作成する
%%time
file_name = "/content/drive/MyDrive/ReazonSpeech.mp4"
results, _ = model.transcribe(file_name,
                              language="ja")

create_txt_file(file_name="transcribe_faster_whisper", results=results, fast_whisper=True)

CPU times: user 1min 23s, sys: 4.07 s, total: 1min 27s
Wall time: 1min 22s


In [ ]:
%%capture
!pip install langchain==0.0.125 openai==0.27.2 tiktoken==0.3.3

from langchain import PromptTemplate, LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
os.environ["OPENAI_API_KEY"] = "***"

In [ ]:
# 文章の校正を行う
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=1000)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
)

# 文字起こしした内容を分割する
with open("transcribe_faster_whisper.txt") as f:
    transcribe_sentence = f.read()
texts = text_splitter.split_text(transcribe_sentence)
docs = [Document(page_content=t) for t in texts]

In [ ]:
template = """
次の文章は動画の文字起こしです。句読点などを適宜追加して文章を校正してください。
また、無駄に長い文章は、意味が変わらない範囲で短く修正してください。
文章：{text}
校正された文章：
"""

PROMPT = PromptTemplate(
    input_variables=["text"],
    template=template,
)

llm_chain = LLMChain(
    prompt=PROMPT, 
    llm=llm
)

In [ ]:
# 文章を校正して、テキストファイルを作成する
from tqdm import tqdm

proofread_sentences = []
for i in tqdm(range(len(texts))):
    proofread_sentences.append(llm_chain.run(texts[i]))

with open("proofread.txt", mode="w") as f:
    for text in proofread_sentences :
        print_text = text.replace("\n\n","\n")
        f.write(f'{print_text}')

100%|██████████| 4/4 [01:37<00:00, 24.33s/it]


In [ ]:
# langchainを使って要約する場合

# 校正された文章を分割する
with open("proofread.txt") as f:
    transcribe_sentence = f.read()
texts = text_splitter.split_text(transcribe_sentence)
docs = [Document(page_content=t) for t in texts]

In [ ]:
template = """
次の文章を日本語で簡潔に要約してください。
文章：{text}
"""

PROMPT = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=PROMPT, combine_prompt=PROMPT, verbose=True)
summary = chain.run(docs)



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:

次の文章を日本語で簡潔に要約してください。
文章：はい、みなさんこんにちは。AIブチマニアタです。
本日は、高精度の文字起こしができるレアゾンスピッチを紹介していこうと思います。
AIで高精度の文字起こしができるモデルとしては、オープンAIのウィスパーが有名なんですけども、先日日本語に特化したレアゾンスピッチというものが公開されました。
この動画では、レアゾンスピッチを簡単に紹介した後に、実際に文字起こしをやってみたいと思います。
誰でも使えるように、Googleコラボのリンクを貼っていますので、無料で高精度の文字起こしを試してみたい人や、最新の音声認識AIについて興味がある人は、ぜひ見ていってもらえればと思います。
それでは、早速やっていきましょう。
最初に、レアゾンスピッチの紹介なんですけども、レアゾンスピッチ聞いたことある人はあまりいないんじゃないかなというふうに思います。
レアゾンスピッチっていうのは、株式会社レアゾンで開発された音声認識に関するモデルやデータのことを指しています。
日本語の音声データや最新のブログなども公開されているので、興味がある人は概要欄にリンクを貼っていくので、見ていってもらえればと思います。
今回紹介するレアゾンスピッチの音声認識モデルは、1セグ放送の録画データから取得した19,000時間のデータを学習させているということで、日本語に特化した音声認識モデルというふうに言えるかと思います。
さらに、今現在も開発中で、年間1万時間以上学習データが増えているということなので、今後さらに日本語に対する文字起こしの精度向上が期待できそうです。
また、ありがたいことに、モデルは商用利用可能で無償で公開されています。
次に、精度の話になります。
こちらは、各データセットに対する誤判定を表す指標になっていて、小さいほど精度が高いことを意味しています。
これまで世界最高精度を持っているのは、ウィスパーのLarge V2というモデルで、グラフの中では、この網網の棒グラフのところですね。
今回紹介するレアゾーンスピーチのNextというものは、この青い棒グラフになってまして、ウィスパーのLarge V2より



> Entering new LLMChain chain...
Prompt after formatting:

次の文章を日本語で簡潔に要約してください。
文章：本日は、日本語に特化した音声認識モデル「レアゾンスピッチ」を紹介し、Googleコラボのリンクを貼って無料で高精度の文字起こしを試せることを伝えました。レアゾンスピッチは、19,000時間の1セグ放送の録画データから学習され、精度はウィスパーのLarge V2よりも高いとされています。また、パラメーターの数が小さい点も特徴的です。商用利用可能で無償で公開されているため、今後の日本語に対する文字起こしの精度向上が期待されます。

グラフには、パラメーターの大きさと誤判定率が表示されており、レアゾーンスピーチは比較的大規模な計算機がなくても動作することができる。Googleコラボを使用する場合、GPUを使用することで高速に文字起こしを行うことができる。Googleコラボのセルを実行することで、レアゾーンスピーチをインストールし、必要なパッケージを選択することができる。

Google Colab上にレアゾーンスピーチのモデルをインストールし、文字起こししたい動画をアップロードして前処理を行い、クリエイトファイルでテキストかsrtファイルを指定して実行すると、文字起こしが終わる。

文字起こしの方法を説明し、トランスクライブテキストを使用することで、日本語の音声データを学習するモデルが非常に良いと述べている。しかし、一部の長さや間違いがあるため、サンプルレートを調整する必要がある。また、長い動画に対してどのように機能するかや、ウィスパーと比較して予測精度がどの程度良いかについては、まだ検証されていない。

コメント欄で意見を教えてほしいと思います。以上で、レアゾンスピーチに関する紹介の動画を終わります。良かったら高評価、コメント、チャンネル登録をお願いします。また次回の動画で会いましょう。




> Finished chain.

> Finished chain.


In [ ]:
print(summary.replace("。", "。\n"))

本日は、日本語に特化した音声認識モデル「レアゾンスピッチ」を紹介し、Googleコラボのリンクを貼って無料で高精度の文字起こしを試せることを伝えました。
レアゾンスピッチは、19,000時間の1セグ放送の録画データから学習され、精度はウィスパーのLarge V2よりも高いとされています。
商用利用可能で無償で公開されているため、今後の日本語に対する文字起こしの精度向上が期待されます。
Googleコラボを使用することで、高速に文字起こしを行うことができます。
ただし、長い動画に対してどのように機能するかや、予測精度がどの程度良いかについては、まだ検証されていません。

